In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib

import sklearn
from sklearn import *

/Users/saumik/mambaforge/envs/random2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
%matplotlib inline
sns.set_theme()

In [4]:
# sklearn.ensemble.RandomForestClassifier
# sklearn.ensemble.RandomForestRegressor

# sklearn.linear_model.LogisticRegression
# sklearn.linear_model.LinearRegression

# sklearn.tree.DecisionTreeClassifier
# sklearn.tree.DecisionTreeRegressor

# sklearn.svm.LinearSVC
# sklearn.svm.LinearSVR

# sklearn.neighbors.KNeighborsClassifier
# sklearn.neighbors.KNeighborsRegressor

In [5]:
# Load data and preprocess (taken from 4-data_cleaning.ipynb)
df = pd.read_csv('titanic.csv')
df = df.drop(columns='alive')
df.fillna(df.mean(numeric_only=True), inplace=True) # Fill numeric NaNs with column mean
df.fillna(df.mode().iloc[0], inplace=True) # Fill nonnumeric NaNs with column mode
numeric_cols = df.select_dtypes(include=['number']).columns.drop('survived')
df[numeric_cols] = df[numeric_cols].apply(lambda x: (x-x.mean())/x.std()) # Standardize all numeric columns except Y
df = pd.get_dummies(df, columns=df.columns[df.dtypes=='object'], drop_first=True) # OneHotEncode all nonnumeric columns
df

,survived,pclass,age,sibsp,parch,fare,adult_male,alone,sex_male,embarked_Q,...,who_man,who_woman,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,embark_town_Queenstown,embark_town_Southampton
0,0,0.826913,-0.592148,0.432550,-0.473408,-0.502163,True,False,True,False,...,True,False,False,True,False,False,False,False,False,True
1,1,-1.565228,0.638430,0.432550,-0.473408,0.786404,False,False,False,False,...,False,True,False,True,False,False,False,False,False,False
2,1,0.826913,-0.284503,-0.474279,-0.473408,-0.488580,False,True,False,False,...,False,True,False,True,False,False,False,False,False,True
3,1,-1.565228,0.407697,0.432550,-0.473408,0.420494,False,False,False,False,...,False,True,False,True,False,False,False,False,False,True
4,0,0.826913,0.407697,-0.474279,-0.473408,-0.486064,True,True,True,False,...,True,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,-0.369158,-0.207592,-0.474279,-0.473408,-0.386454,True,True,True,False,...,True,False,False,True,False,False,False,False,False,True
887,1,-1.565228,-0.822881,-0.474279,-0.473408,-0.044356,False,True,False,False,...,False,True,True,False,False,False,False,False,False,True
888,0,0.826913,0.000000,0.432550,2.007806,-0.176164,False,False,False,False,...,False,True,False,True,False,False,False,False,False,True
889,1,-1.565228,-0.284503,-0.474279,-0.473408,-0.044356,True,True,True,False,...,True,False,False,True,False,False,False,False,False,False


In [6]:
Y, X = df.survived, df.drop(columns='survived')
Y, X = Y.values, X.values.astype(float)
Y.shape, X.shape

((891,), (891, 22))

In [7]:
scoring = {'acc': 'accuracy',
           'f1': 'f1',
           'auc': 'roc_auc'}


In [8]:
estimator = sklearn.neighbors.KNeighborsClassifier()
res = sklearn.model_selection.cross_validate(estimator, X=X, y=Y, scoring=scoring)
display(res)
display({k:v.mean() for k,v in res.items()})

{'fit_time': array([0.00233912, 0.00128794, 0.00090885, 0.00068688, 0.00070119]),
 'score_time': array([0.20885992, 0.00809097, 0.00847101, 0.0085721 , 0.00929904]),
 'test_acc': array([0.79888268, 0.78651685, 0.82022472, 0.83146067, 0.84831461]),
 'test_f1': array([0.71875   , 0.71641791, 0.75384615, 0.76190476, 0.79699248]),
 'test_auc': array([0.84057971, 0.80040107, 0.86898396, 0.87486631, 0.88106635])}

{'fit_time': 0.0011847972869873046,
 'score_time': 0.04865860939025879,
 'test_acc': 0.817079907099366,
 'test_f1': 0.7495822614803369,
 'test_auc': 0.8531794789206977}

In [57]:
# Optional, reduce features by performing PCA
# pca = sklearn.decomposition.PCA(n_components=10)
# newX = pca.fit_transform(X)

# estimator = sklearn.neighbors.KNeighborsClassifier()
# res = sklearn.model_selection.cross_validate(estimator, X=newX, y=Y)
# display(res)
# display({k:v.mean() for k,v in res.items()})

In [9]:
estimator = sklearn.ensemble.RandomForestClassifier()
res = sklearn.model_selection.cross_validate(estimator, X=X, y=Y, scoring=scoring)
display(res)
display({k:v.mean() for k,v in res.items()})

{'fit_time': array([0.235569  , 0.20223498, 0.2073679 , 0.20653319, 0.20857406]),
 'score_time': array([0.02844596, 0.02703404, 0.02702999, 0.02668285, 0.02802801]),
 'test_acc': array([0.77653631, 0.78089888, 0.84269663, 0.75280899, 0.85955056]),
 'test_f1': array([0.71428571, 0.71111111, 0.78787879, 0.64516129, 0.81751825]),
 'test_auc': array([0.83274045, 0.80086898, 0.89745989, 0.84625668, 0.89110491])}

{'fit_time': 0.21205582618713378,
 'score_time': 0.027444171905517577,
 'test_acc': 0.8024982738057874,
 'test_f1': 0.7351910303546753,
 'test_auc': 0.8536861831435261}

In [55]:
estimator = sklearn.neighbors.KNeighborsClassifier()
estimator.fit(X,Y);